In [ ]:
import coiled
from dask.distributed import Client
import dask

In [ ]:
import dask.dataframe as dd

In [ ]:
cluster = coiled.Cluster(n_workers = 20)
client = cluster.get_client()

In [ ]:
client

In [ ]:
dask.config.set({"dataframe.convert-string": True})

In [ ]:
df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/",
    storage_options={'anon': True}
)

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
dask.utils.format_bytes(
    df.memory_usage(deep=True).sum().compute()
)

In [ ]:
df_sample = df.sample(frac=0.1)

In [ ]:
df_sample = df_sample.persist()

In [ ]:
dask.utils.format_bytes(
    df_sample.memory_usage(deep=True).sum().compute()
)

In [ ]:
from dask.sizeof import sizeof

In [ ]:
partitions_mem_stats = df_sample.map_partitions(sizeof).compute()

In [ ]:
(partitions_mem_stats / 1024**2).describe() #in MiB

In [ ]:
dask.utils.format_bytes(df_sample.partitions[0].memory_usage(deep=True).compute().sum())

In [ ]:
%%time
#passanger fare
df_sample.base_passenger_fare.sum().compute() / 1e9

In [ ]:
%%time
#tip
df_sample.tips.sum().compute() / 1e6

In [ ]:
%%time
df_sample.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

## Partition size 1MB 

Runs are ~11X slower compared to 13MB partitions

In [ ]:
df_sample = df_sample.repartition(partition_size="1MB").persist()

In [ ]:
dask.utils.format_bytes(df_sample.partitions[0].memory_usage(deep=True).compute().sum())

In [ ]:
%%time
#passanger fare
df_sample.base_passenger_fare.sum().compute() / 1e9

In [ ]:
%%time
#tip
df_sample.tips.sum().compute() / 1e6

In [ ]:
%%time
df_sample.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

## Write 1MB partition data to parquet and csv

In [ ]:
df_sample.to_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/");

In [ ]:
df_sample.to_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/");

## Check read 

In [ ]:
df_pq = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/")

In [ ]:
#beauty of parquet meta - get dtypes right
df_pq.dtypes

In [ ]:
dask.utils.format_bytes(df_pq.partitions[0].memory_usage(deep=True).compute().sum())

### Try 100MB partitions

In [ ]:
df_pq = df_pq.repartition(partition_size="100MB").persist()

In [ ]:
%%time
#passanger fare
df_pq.base_passenger_fare.sum().compute() / 1e9

## Went from ~5.9 s to 137ms (~43X faster)

In [ ]:
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
                     dtype = {
                     "shared_request_flag": "category",
                     "shared_match_flag": "category",
                     "access_a_ride_flag": "category",
                     "wav_request_flag": "category",
                     "wav_match_flag": "category",    
                     }, 
                     parse_dates=["request_datetime", "on_scene_datetime", # this is not working well see error below
                                  "pickup_datetime", "dropoff_datetime"]
                    )

In [ ]:
#notice the ints and floats are infered as 64
df_csv.dtypes

In [ ]:
#this is not working
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)

```python
ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

The following columns failed to properly parse as dates:

- on_scene_datetime

This is usually due to an invalid value in that column. To
diagnose and fix it's recommended to drop these columns from the
`parse_dates` keyword, and manually convert them to dates later
using `dd.to_datetime`.


```

## If we just do 
```python
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*",)
```

It will be infering date times and categories as strings, and the last category `wav_match_flag` because it has NaN is converting it to floats but the values are ["N", "Y"]

and this fails because of the dtype of `wav_match_flag`
```python
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)
```